In [2]:
# Import required packages
# We use cobrapy to create the minimal cell
import cobra


In [88]:
model = cobra.Model("minimal_cell")
mets = {}
for m_id in ["a_e", "a_c", "c_e", "c_c", "b_c", "d_c"]:
    mets[m_id] = cobra.Metabolite(m_id, compartment = m_id.split("_")[1])
model.add_metabolites(mets.values())

In [89]:
Ea = cobra.Reaction("Ea")
Ea.add_metabolites({mets["a_e"]:-1})

Ec = cobra.Reaction("Ec")
Ec.add_metabolites({mets["c_e"]:-1})

Ta = cobra.Reaction("Ta")
Ta.add_metabolites({mets["a_e"]:-1, mets["a_c"]:1})
Ta.bounds = (0,1000)

Tc = cobra.Reaction("Tc")
Tc.add_metabolites({mets["c_e"]:-1, mets["c_c"]:1})
Tc.bounds = (0,1000)

R1 = cobra.Reaction("R1")
R1.add_metabolites({mets["a_c"]:-1, mets["b_c"]:1})
R1.bounds = (0,1000)

R2 = cobra.Reaction("R2")
R2.add_metabolites({mets["b_c"]:-1, mets["c_c"]:2})
R2.bounds = (-1000,1000)

R3 = cobra.Reaction("R3")
R3.add_metabolites({mets["b_c"]:-1, mets["c_c"]:-2, mets["d_c"]:1})
R3.bounds = (0,1000)

Growth = cobra.Reaction("Growth")
Growth.add_metabolites({mets["d_c"]:-1})
Growth.bounds = (0,1000)

model.add_reactions([Ea, Ec, Ta, Tc, R1, R2, R3, Growth])


In [90]:
cobra.util.array.create_stoichiometric_matrix(model)

array([[-1.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0., -1.,  0.,  0.,  0.],
       [ 0., -1.,  0., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  2., -2.,  0.],
       [ 0.,  0.,  0.,  0.,  1., -1., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1., -1.]])

In [99]:
# Set the bound of the exchange reaction to define growth environment
model.reactions.Ea.bounds = (-1, 0)
model.reactions.Ec.bounds = (-3, 0)
model.reactions.R2.bounds

(-1000, 1000)

In [100]:
# Set objctive
model.objective = model.reactions.Growth


In [101]:
# Optimize
model.optimize()
print(model.summary(float_format = None))

IN_FLUXES IN_FLUXES OUT_FLUXES OUT_FLUXES OBJECTIVES OBJECTIVES
    ID       FLUX       ID        FLUX        ID        FLUX   
    c_e       3.0                           Growth      1.25   
    a_e       1.0                                              


In [102]:
for r in model.reactions:
    print(r.id, r.flux)

Ea -1.0
Ec -3.0
Ta 1.0
Tc 3.0
R1 1.0
R2 -0.25
R3 1.25
Growth 1.25


In [104]:
cobra.io.save_json_model(model, "../minimal_cell.json")